<a href="https://colab.research.google.com/github/Gabriela2685/Accidentes/blob/main/puntos_negros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import folium
import gc

In [ ]:
data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQGVuLESo9dj1QzZuPoqvsQBvGT3-NIjLA34Mus7Q6yPAxvQpRbWhjsuUJoAUEThw/pub?output=csv")

hora_evento


<ipython-input-22-65710d555c13>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQGVuLESo9dj1QzZuPoqvsQBvGT3-NIjLA34Mus7Q6yPAxvQpRbWhjsuUJoAUEThw/pub?output=csv")


In [ ]:
# Imprimir el número de filas
print(f"Número de filas: {len(data)}")

Número de filas: 67366


In [ ]:
# Eliminar filas con valores NaN en latitud y longitud
data = data.dropna(subset=['latitud', 'longitud'])

# Tomar una muestra de 10,000 filas para reducir el consumo de memoria
data_sample = data.sample(n=20000, random_state=42)

# Crear un array con las coordenadas de la muestra
coordinates = data_sample[['latitud', 'longitud']].to_numpy()

# Configurar y aplicar el modelo MiniBatchKMeans para clústeres
num_clusters = 50  # Define el número de clústeres
kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=10000, random_state=42)
data_sample['cluster'] = kmeans.fit_predict(coordinates)

# Liberar memoria no utilizada
gc.collect()

# Visualizar el número de clústeres encontrados
print(f"Número de clústeres especificados: {num_clusters}")

# Calcular el centro de cada clúster
from shapely.geometry import MultiPoint
centroids = []
for cluster_num in range(num_clusters):
    cluster_points = coordinates[data_sample['cluster'] == cluster_num]
    cluster_center = MultiPoint(cluster_points).centroid
    centroids.append(cluster_center)

print("Centroides de los clústeres:", centroids)

# Crear un mapa centrado en la Ciudad de México
mapa = folium.Map(location=[19.4326, -99.1332], zoom_start=12)

# Agregar los puntos de accidentes al mapa
for i, row in data_sample.iterrows():
    folium.CircleMarker(
        location=(row['latitud'], row['longitud']),
        radius=5, color='blue', fill=True, fill_color='blue'
    ).add_to(mapa)

# Agregar los centroides al mapa (puntos rojos)
for centroide in kmeans.cluster_centers_:
    latitud, longitud = centroide  # Extraer las coordenadas del centroide
    folium.CircleMarker(
        location=(latitud, longitud),  # latitud, longitud del centroide
        radius=7, color='red', fill=True, fill_color='red'
    ).add_to(mapa)

# Guardar o mostrar el mapa
mapa.save("mapa_accidentes3.html")

Número de clústeres especificados: 50
Centroides de los clústeres: [<POINT (19.37 -99.171)>, <POINT (19.425 -99.15)>, <POINT (19.448 -99.099)>, <POINT (19.286 -99.122)>, <POINT (19.366 -99.024)>, <POINT (19.441 -99.172)>, <POINT (19.376 -99.244)>, <POINT (19.413 -99.085)>, <POINT (19.277 -99.006)>, <POINT (19.533 -99.15)>, <POINT (19.302 -99.215)>, <POINT (19.387 -99.141)>, <POINT (19.479 -99.193)>, <POINT (19.316 -99.076)>, <POINT (19.467 -99.151)>, <POINT (19.372 -99.11)>, <POINT (19.251 -99.098)>, <POINT (19.294 -99.168)>, <POINT (19.394 -99.187)>, <POINT (19.331 -99.111)>, <POINT (19.407 -99.168)>, <POINT (19.488 -99.087)>, <POINT (19.432 -99.119)>, <POINT (19.205 -98.987)>, <POINT (19.321 -99.325)>, <POINT (-19.327 -99.114)>, <POINT (19.338 -99.17)>, <POINT (19.472 -99.117)>, <POINT (19.338 -99.042)>, <POINT (19.356 -99.143)>, <POINT (19.322 -99.248)>, <POINT (19.432 -99.206)>, <POINT (19.411 -99.131)>, <POINT (19.455 -99.068)>, <POINT (19.389 -99.05)>, <POINT (19.354 -99.068)>, <